In [1]:
import torch
import os
from ADMM_ConvexGCN import Net
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataListLoader
from ADMM_ConvexGCN import ResumeModel, save_recurrencePlots
from pyts.image import RecurrencePlot 
from utils.weight_norm import weight_norm

ModuleNotFoundError: No module named 'torch_scatter'

In [3]:
import torch.nn as nn
import torch.optim as optim

In [4]:
from quantization import ADMM_quantization

In [5]:
model_name="GCN"
dataset="Cora"
params=[512,0.2]
Monte_iter=0

In [6]:
root='/data/GraphData/'+dataset
Batch_size=params[0]
datasetroot = Planetoid(root=root, name=dataset).shuffle()
trainloader = DataListLoader(datasetroot, batch_size=Batch_size, shuffle=True)
testloader = DataListLoader(datasetroot, batch_size=100, shuffle=False)
hessianloader = DataListLoader(datasetroot, batch_size=2, shuffle=False)

In [105]:
#model_to_save='./checkpoint/{}-{}-param_{}_{}-Mon_{}-ckpt.pth'.format(dataset,model_name,params[0],params[1],Monte_iter)
model_to_save='./%s/%s_%s_%s_%s_pretrain.pth' %(model_name, dataset, model_name,params[0],params[1])


In [107]:
torch.load(model_to_save).state_dict()

AttributeError: 'collections.OrderedDict' object has no attribute 'state_dict'

In [ ]:
[net,TrainConvergence,TestConvergence,start_epoch]=ResumeModel(model_to_save)
optimized_net=net
state_dict = optimized_net.state_dict()
state_dict["conv4.bias"]

In [102]:
net.module.state_dict()

AttributeError: 'Net' object has no attribute 'module'

In [8]:
device = torch.device('cuda')

In [9]:
from ADMM_ConvexGCN import ComputeHessian

In [10]:
for data_list in trainloader:
    output = net(data_list)
    y = torch.cat([data.y for data in data_list]).to(output.device)
    loss = criterion(output, y)
    hessian_matrix=hessian(loss,data_list,create_graph=True)

AttributeError: 'list' object has no attribute 'x'

In [91]:
from utils.weight_norm import remove_weight_norm
from utils.prune import ln_structured, remove

In [94]:
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
import numpy as np
import skcuda.linalg as linalg
from skcuda.linalg import PCA as cuPCA


ModuleNotFoundError: No module named 'skcuda'

In [96]:
pca = cuPCA(n_components=4) # map the data to 4 dimensions
X = np.random.rand(1000,100) # 1000 samples of 100-dimensional data vectors
X_gpu = gpuarray.GPUArray((1000,100), np.float64, order="F") # note that order="F" or a transpose is necessary. fit_transform requires row-major matrices, and column-major is the default
X_gpu.set(X) # copy data to gpu
T_gpu = pca.fit_transform(X_gpu) # calculate the principal components
linalg.dot(T_gpu[:,0], T_gpu[:,1]) # show that the resulting eigenvectors are orthogonal

NameError: name 'cuPCA' is not defined

In [108]:
for layer_name, parameters in optimized_net.named_parameters():
    print(parameters.shape)


torch.Size([1433, 116])
torch.Size([116])
torch.Size([116, 104])
torch.Size([104])
torch.Size([104, 53])
torch.Size([53])
torch.Size([53, 7])
torch.Size([7])


In [113]:
for name, module in optimized_net.named_modules():
    print(name)

 fgg
conv1 fgg
conv2 fgg
conv3 fgg
conv4 fgg


In [98]:
m = weight_norm(net.module)
remove_weight_norm(m)

AttributeError: 'Net' object has no attribute 'module'

In [99]:
net.module.state_dict()['%s.weight' % layer_name].size()

AttributeError: 'Net' object has no attribute 'module'

In [100]:
net.module.state_dict()['%s.weight' % layer_name].size()

AttributeError: 'Net' object has no attribute 'module'

In [15]:
for layer_name, parameters in optimized_net.named_parameters():
        #prune.l1_unstructured(parameters,layer_name,0.6)
        m = weight_norm(parameters)
        remove_weight_norm(m)
        

AttributeError: 'Parameter' object has no attribute '_forward_pre_hooks'

In [32]:
m = weight_norm(nn.Linear(20, 40))
remove_weight_norm(m)
print(nn.Linear(29,24))

Linear(in_features=29, out_features=24, bias=True)


In [44]:
type(nn.Conv2d(5, 3, 2))
import numpy as np

In [83]:
a=np.random.random((200,4))

In [84]:
b=np.ones((200,10))

In [87]:
X=np.concatenate((a,b),axis=1)

In [89]:
from factor_analyzer import FactorAnalyzer
transformer=FactorAnalyzer(n_factors=3)

In [90]:
X_transformed = transformer.fit_transform(X)
X_transformed.shape

/opt/conda/lib/python3.6/site-packages/factor_analyzer/utils.py:91: RuntimeWarning: invalid value encountered in true_divide
  x = (x - x.mean(0)) / x.std(0)


ValueError: array must not contain infs or NaNs

In [38]:
for layer_name, parameters in optimized_net.named_parameters():
        #prune.l1_unstructured(parameters,layer_name,0.6)
        ln_structured(
               nn.Linear(20, 40), layer_name, 0.5,dim=1, n=float('-inf'))
        remove()

AttributeError: 'Linear' object has no attribute 'conv1.weight'

In [16]:
optimized_net.modules[conv1]

<bound method Module.modules of Net(
  (conv1): GCNConv(1433, 116)
  (conv2): GCNConv(116, 104)
  (conv3): GCNConv(104, 53)
  (conv4): GCNConv(53, 7)
)>

In [ ]:
    hessian = generate_hessian(quantized_net, hessianloader, layer_name, layer_type='F')
    updated_weight = state_dict[layer_name].cpu().numpy() 
    updated_bias = state_dict[layer_name].cpu().numpy() 
    quantized_weight, quantized_bias = ADMM_quantization(layer_name=layer_name, layer_type='F', kernel=updated_weight, bias=updated_bias,
                                                         hessian=hessian, kbits=2)

In [139]:
import torch
from hessian import hessian

x = torch.tensor([1.5, 2.5], requires_grad=True)
h = hessian(x.sum(), x, create_graph=True)


TypeError: 'module' object is not callable

In [13]:
from prune import BasePruningMethod,L1PruningMethod


In [1]:
import utils.prune as  prune

In [16]:
import numpy as np

def hessian(x):
    xgrad = np.gradient(x) 
    hessian = np.empty((x.ndim, x.ndim) + x.shape, dtype=x.dtype) 
    for k, grad_k in enumerate(xgrad):
        tmp_grad = np.gradient(grad_k) 
        for l, grad_kl in enumerate(tmp_grad):
            hessian[k, l, :, :] = grad_kl
    return hessian

if __name__=="__main__":
    x = np.random.randn(100, 100, 100)
    hessian(x)

In [11]:
torch.nn.utils.pruning_function?

Object `torch.nn.utils.pruning_function` not found.
